<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Editado por Equipo Docente IIC2233 2018-1, 2018-2.</font>
</p>

# Qt Designer

Cuando las interfaces gráficas poseen pocos _widgets_ resulta sencillo crearlas manualmente agregando cada uno de ellos mediante líneas de código. Sin embargo, cuando la interfaz incluye un mayor número de objetos, interacciones o controles, la labor de codificarlos manualmente no es tan rápido. PyQt incluye una herramienta llamada **Qt Designer** que permite construir la interfaz gráfica visualmente. Qt Designer, además de crear cada _widget_ de la interfaz, permite tener leer y modificar las propiedades de los _widgets_. La siguiente figura muestra un ejemplo de la ventana principal de Qt Designer.

![](img/pyqt-qt-designer.png)

Todas las acciones de Qt Designer pueden ser controladas desde los menús de la aplicación. Existen 4 secciones importantes en la ventana de trabajo. Al lado izquierdo se encuentra el **Widget Box** que contiene todos los _widgets_ ordenados por tipo que pueden ser agregados al formulario. Al lado derecho se encuentra primero el **Inspector de objetos** que permite desplegar visualmente la jerarquía de _widgets_ que existen en el formulario. A continuación del _Inspector de Objetos_ se encuentra el **Editor de propiedades**. Finalmente, en el centro, se encuentra el _widget central_ que puede ser un formulario, una ventana simple, o algún _widget_ más complejo.

![](img/qt-designer-tools.png)

Una vez que un _widget_ es agregado a la interfaz, se le asigna un nombre por defecto al campo `objectName` del editor de propiedades. Este nombre será el que usaremos para referenciar al objeto desde el código Python que maneja la interfaz, y puede ser cambiado directamente en Qt Designer, como muestra la siguiente figura. 

![](img/qt-designer-property-editor.png)

Una forma rápida de ver el resultado, sin agregarlo en el código final, es usar el modo _preview_ de Qt Designer pulsando `Ctrl + R`. Como resultado podemos ver la interfaz creada, pero solo como _front-end_. Una vez que concluimos con la creación de la interfaz, debemos guardarla en un archivo con extensión `.ui`. Finalmente, debemos ensamblarla en el programa en Python que contendrá el resto de las funcionalidades de los _widgets_ incluidos en la interfaz.

Para incorporar la interfaz a nuestro código debemos utilizar el módulo ``uic``. Este módulo nos permite cargar la interfaz mediante el método `loadUiType(<nombre-interfaz.ui>)`. Este método retorna una tupla con dos elementos. El primero corresponde a una clase con el nombre de la ventana definida en la propiedad `objectName` de Qt Designer. El segundo corresponde al nombre de la clase de la cual hereda los comportamientos.

![](img/qt-designer-window-name.png)

El siguiente ejemplo muestra como realizar este procedimiento usando una interfaz previamente creada. En este caso, al invocar `uic.loadUiType(<archivo.ui>)`, obtenemos una tupla `(class 'Ui_MainWindow', class 'PyQt5.QtGui.QMainWindow')`. El prefijo **Ui** asociado al nombre de la clase que contiene la interfaz es asignado por el módulo `uic` durante la carga de la interfaz. Una vez que hemos cargado la interfaz, debemos inicializarla dentro del método `__init()__` de la clase que hereda la interfaz. Esta inicialización se realiza mediante el método `setupUi(self)` que recibe como parámetro la misma instancia.

La creación de la aplicación debe ser realizada usando la estructura del programa principal `main` visto durante el comienzo de la explicación de interfaces gráficas.

In [ ]:
import sys
from PyQt5 import uic
from PyQt5.QtWidgets import QApplication

"""
Creamos nuestra ventana principal heredando desde la GUI creada con Designer.
La función loadUiType retorna una tupla en donde el primer elemento
corresponde al nombre de la ventana definido en QtDesigner, y el segundo
elemento a la clase base de la GUI.
"""

window_name, base_class = uic.loadUiType("qt-designer-label.ui")


class MainWindow(window_name, base_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)


if __name__ == '__main__':
    app = QApplication([])
    form = MainWindow()
    form.show()
    sys.exit(app.exec_())


La siguiente figura muestra el resultado al usar la interfaz creada en Qt Designe_ y luego embeberla en un programa en Python.

![](img/qt-designer-label.png)

Los comportamientos de cada _widget_ deben ser definidos mediante _signals_ y _slots_ tal como se explicó anteriormente. A continuación se muestra un ejemplo con una interfaz que realiza la división entre dos números. El botón creado en Qt Designer despliega el resultado sobre una etiqueta.

![](img/qt-designer-division-ui.png)

In [ ]:
import sys
from PyQt5 import uic
from PyQt5.QtWidgets import (QApplication, QMessageBox)

# Cargamos el formulario usando uic
window_name, base_class = uic.loadUiType("qt-designer-mainwindow.ui")


class MainWindow(window_name, base_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)

        """Creamos las conexiones con los puertos."""
        self.pushButton1.clicked.connect(self.click_button)

    def click_button(self):
        """
        Este método controla la acción ejecuta cada vez que presionamos el
        botón1.
        """

        try:
            self.label_3.setText('= ' + str(
                float(self.lineEdit1.text()) / float(self.lineEdit2.text())))
        except ValueError as err:
            """Existen cuadros de diálogo pre-construidos. En este caso
            usaremos un MessageBox para mostrar el mensaje de error.
            """
            QMessageBox.warning(self, '', str(err))


if __name__ == '__main__':
    app = QApplication([])
    form = MainWindow()
    form.show()
    sys.exit(app.exec_())


En _Qt Designer_ es fácil incluir _widgets_ que hacen más fácil la interacción del usuario con la interfaz. Un ejemplo es el **Radio Button**, que permite capturar opciones del usuario en el formulario. Una ventaja que presenta este _widget_ es que disminuye la ambigüedad de la información ingresada por los usuarios. Por defecto, los _radio buttons_ de un formulario son autoexcluyentes, es decir, puede haber solo uno seleccionado. La siguiente figura muestra un ejemplo del diseño de formularios utilizando _radio buttons_ y el código en Python utilizado para poder verificar los valores.

![](img/qt-designer-radiobutton.png)

In [ ]:
import sys

from PyQt5 import uic
from PyQt5.QtWidgets import QApplication

window_name, base_class = uic.loadUiType("qt-designer-radiobutton.ui")
print(window_name, base_class)


class MainWindow(window_name, base_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.pushButton1.clicked.connect(self.mostrar_gustos)

    def mostrar_gustos(self):
        for rb_id in range(1, 3):
            if getattr(self, 'radioButton' + str(rb_id)).isChecked():
                opcion = getattr(self, 'radioButton' + str(rb_id)).text()
                self.label2.setText(f'prefiere: {opcion}')


if __name__ == '__main__':
    app = QApplication([])
    form = MainWindow()
    form.show()
    sys.exit(app.exec_())


El estado de un *radio button* puede ser verificado usando el método ```isChecked()``` y la información que despliega corresponde al texto, que puede ser retornado mediante el método ```text()```.